**<h1>A. Application </h1>**

Créer une **application chatbot avec gestion de sessions** :
- **Frontend** : Interface utilisateur avec Streamlit.
- **Backend** : Répond à l'utilisateur via une API FastAPI.

---

**<h2> 🛠️ 1. Installation des bibliothèques </h2>**

Installer les bibliothèques nécessaires dans un environnement virtuel (ou globalement) avec pip :

```bash
pip install streamlit fastapi uvicorn requests
```

---

**<h2> 📁 2. Création du fichier `frontend.py`</h2>**

Ce fichier contient **l’interface utilisateur** via Streamlit.

> 📄 Créez un fichier appelé `frontend.py` et collez le code ci-dessous :

```python
import streamlit as st
import requests
import time
from requests.utils import quote

# Fonction qui envoie le prompt à l'API backend et récupère la réponse
def response_generator(prompt):
    encoded_prompt = quote(prompt, safe="")  # Encode le prompt pour éviter les erreurs d'URL
    response = requests.get(f"http://localhost:8000/llm/{encoded_prompt}")
    response_text = response.json().get("answer", "Erreur : aucune réponse reçue.")

    # Génère la réponse mot par mot avec un petit délai pour effet "chat"
    for word in response_text.split():
        yield word + " "
        time.sleep(0.05)

# Titre principal
st.title("Chatbot avec gestion des sessions")

# Initialisation de la variable de session si elle n'existe pas
if "sessions" not in st.session_state:
    st.session_state.sessions = {"Session 1": []}  # Une session par défaut
if "current_session" not in st.session_state:
    st.session_state.current_session = "Session 1"

# Barre latérale pour choisir ou créer des sessions
st.sidebar.title("Sessions de Chat")
session_names = list(st.session_state.sessions.keys())
selected_session = st.sidebar.selectbox("Sélectionnez une session", session_names)
st.session_state.current_session = selected_session

# Affiche l'historique de la session sélectionnée
st.subheader(f"Chat - {st.session_state.current_session}")
for message in st.session_state.sessions[selected_session]:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Créer une nouvelle session
if st.sidebar.button("Créer une nouvelle session"):
    new_session_name = f"Session {len(st.session_state.sessions) + 1}"
    st.session_state.sessions[new_session_name] = []
    st.session_state.current_session = new_session_name
    st.rerun()  # Recharge la page avec la nouvelle session

# Zone d'entrée utilisateur
if prompt := st.chat_input("Que voulez-vous dire ?"):
    # Ajoute le message utilisateur
    st.session_state.sessions[st.session_state.current_session].append(
        {"role": "user", "content": prompt}
    )
    with st.chat_message("user"):
        st.markdown(prompt)

    # Appelle le backend et affiche la réponse
    with st.chat_message("assistant"):
        response = st.write_stream(response_generator(prompt))

    # Sauvegarde la réponse dans la session
    st.session_state.sessions[st.session_state.current_session].append(
        {"role": "assistant", "content": response}
    )
```

---

**<h2> ⚙️ 3. Création du fichier `backend.py`</h2>**

> 📄 Créez un fichier `backend.py` avec ce code :

```python
from fastapi import FastAPI
import uvicorn

app = FastAPI()

# Route qui prend un prompt et retourne une réponse simulée
@app.get("/llm/{prompt}")
async def get_response(prompt: str):
    return {"answer": f"Hey, My name is Chatbot and you say: {prompt}"}

# Lancer l'API si on exécute le fichier directement
if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=8000)
```

---

**<h2> 🚀 4. Exécution des fichiers </h2>**

Maintenant que tout est prêt, on va lancer les deux parties de l'application :

**<h3> Étape 1 : Lancer le backend FastAPI</h3>**

Dans un terminal, exécute :

```bash
python backend.py
```

Cela lance un serveur API sur `http://localhost:8000`.

**<h3> Étape 2 : Lancer le frontend Streamlit</h3>**

Dans un autre terminal :

```bash
streamlit run frontend.py
```

Cela ouvre automatiquement une interface web (généralement à `http://localhost:8501`).

---

**<h2> ✅ Résultat attendu </h2>**

- Une interface web simple.
- Gestion de plusieurs **sessions de chat**.
- Le chatbot répond dynamiquement à chaque message envoyé.
- Backend simule une réponse mais peut être remplacé plus tard par un vrai modèle IA.

---

**<h2> 🧑‍🏫 Pistes d’approfondissement </h2>**

- Intégrer **HuggingFace** ou **LangChain** dans le backend.
- Sauvegarder les conversations avec une base de données.
- Déployer le frontend avec **[Streamlit Cloud](https://www.youtube.com/watch?v=JL9xOs-G1hI)** et le backend sur **[Render](https://www.youtube.com/watch?v=stDadw38H0I)** ou **Railway**.

**<h1>B. Exécution des LLMs</h1>**

**<h2>1. 🌐 Utiliser un LLM via Hugging Face Inference Client</h2>**

**Avantage** : Simple, rapide, hébergé (pas besoin de GPU local).

 ✅ Étapes :
1. Installe la bibliothèque :
```bash
pip install huggingface_hub
```

2. Code :

```python
from huggingface_hub import InferenceClient
import os

os.environ["HF_TOKEN"] = "TA_CLÉ_HF"

client = InferenceClient("mistralai/Mistral-7B-Instruct-v0.3", token=os.getenv("HF_TOKEN"))

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the meaning of life?"}
]

response = client.chat_completion(messages, max_tokens=1024)
print(response["choices"][0]["message"]["content"])
```

📌 *Utilise les modèles HF comme une API sécurisée.*

---

**<h2> 2. 💻 Utiliser un LLM en local avec Llama.cpp</h2>**

**Avantage** : 100% local, pas besoin d'accès internet ou d’API.

✅ Étapes :

1. Installer la lib et télécharger le modèle :
```bash
pip install llama-cpp-python
wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q2_K.gguf -O mistral.gguf
```

2. Code :

```python
from llama_cpp import Llama

llm = Llama(model_path="mistral.gguf")

output = llm(
   "Q: Who is the president of USA? A: ",
   max_tokens=1024,
   stop=["Q:", "\n"],
   echo=True
)

print(output["choices"][0]["text"])
```

📌 *Idéal pour les apps sans connexion ou avec contraintes de confidentialité.*

---

**<h2> 3. ☁️ Utiliser un LLM via DeepInfra (API Huggingface-compatible)</h2>**

**Avantage** : Interface de type OpenAI, simple, rapide à tester.

✅ Étapes :
1. Crée un compte sur [deepinfra.com](https://deepinfra.com) et récupère ton `api_key`.

2. Code :

```python
import requests

api_key = "TA_CLÉ_DEEPINFRA"
url = "https://api.deepinfra.com/v1/openai/chat/completions"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

data = {
    "model": "mistralai/Mistral-7B-Instruct-v0.1",
    "messages": [
        {"role": "system", "content": "Tu es un assistant utile."},
        {"role": "user", "content": "Comment prévenir le diabète ?"}
    ],
    "temperature": 0.7,
    "max_tokens": 1024
}

res = requests.post(url, headers=headers, json=data)
print(res.json()["choices"][0]["message"]["content"])
```

📌 *Parfait pour prototyper des assistants dans des apps web ou mobiles.*

---

**<h2>🧩 Bonus : Autres APIs possibles</h2>**

- **OpenAI API** (`gpt-3.5`, `gpt-4`) : très puissante, payante.
- **[Replicate](https://replicate.com/)**, **[Together.ai](Together.ai)**, **[Anyscale](https://www.anyscale.com/blog/anyscale-endpoints-fast-and-scalable-llm-apis)** : offrent aussi des modèles open-source via API.
- **LangChain** : pour chaîner des appels de LLM avec logique métier.

---

**<h2>🎓 En résumé</h2>**

| Méthode        | Niveau technique | Avantages                         | Inconvénients                    |
|----------------|------------------|-----------------------------------|----------------------------------|
| Hugging Face   | Facile           | Hébergé, rapide, fiable           | Besoin d’un token                |
| Llama.cpp      | Moyen            | Local, gratuit, confidentiel      | Modèle lourd à télécharger       |
| DeepInfra      | Facile           | Compatible OpenAI, simple à coder | API key nécessaire               |

